In [4]:
from ollama import chat
from pydantic import BaseModel

# Define the schema for the response
class FriendInfo(BaseModel):
  name: str
  age: int
  is_available: bool


class FriendList(BaseModel):
  friends: list[FriendInfo]


class BusinessLocation(BaseModel):
    name: str
    street_number: str
    street_name: str
    city: str
    postal_code: Optional[str] = None
    email: Optional[EmailStr] = None
    website: Optional[HttpUrl] = None

class BusinessLocations(BaseModel):
    addresses: list[BusinessLocation]
# schema = {'type': 'object', 'properties': {'friends': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'age': {'type': 'integer'}, 'is_available': {'type': 'boolean'}}, 'required': ['name', 'age', 'is_available']}}}, 'required': ['friends']}

data_2 = """
[Search Results]
* Ottawa Public Library *
Contact Us > Main Branch
Located at: &nbsp;150 Elgin Street, Ottawa
Status: OPEN NOW! 📚
Customer Service: library@ottawa.ca
Visit us online: https://biblioottawalibrary.ca
Postal: K1P 1L7
---------------------
<div class="business-listing">
Sarah's Bakery & Café [⭐️4.8]
Address line 1: 240 
Address line 2: Laurier Avenue
City: Ottawa
Postal Code: K1P 5J7
Contact: sarah@sarahsbakery.ca
www.sarahsbakery.ca
</div>
...Read More...
---------------------
TECH CORP GLOBAL
www.techcorp.com/contact
📍 1385 Terry Fox Drive
Kanata, Ontario
[Click to view map]
Email: info@techcorp.com
Postal: K2K 3K2
"""

response = chat(
  model='llama3.1:8b',
  messages=[{'role': 'user', 'content': 'I have two friends. The first is Ollama 22 years old busy saving the world, and the second is Alonso 23 years old and wants to hang out. Return a list of friends in JSON format'}],
  format=BusinessLocations.model_json_schema(),  # Use Pydantic to generate the schema or format=schema
  options={'temperature': 0},  # Make responses more deterministic
)

# Use Pydantic to validate the response
friends_response = FriendList.model_validate_json(response.message.content)
print(friends_response)

friends=[FriendInfo(name='Ollama', age=22, is_available=False), FriendInfo(name='Alonso', age=23, is_available=True)]


In [26]:
from typing import Optional, List  # Add List from typing
from pydantic import BaseModel, EmailStr, HttpUrl
from ollama import chat
from pprint import pprint as pp

class BusinessLocation(BaseModel):
    name: str
    street_number: str
    street_name: str
    city: str
    postal_code: str
    email: str
    website: str

class BusinessLocations(BaseModel):
    addresses: list[BusinessLocation]
# schema = {'type': 'object', 'properties': {'friends': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'age': {'type': 'integer'}, 'is_available': {'type': 'boolean'}}, 'required': ['name', 'age', 'is_available']}}}, 'required': ['friends']}


data = """
[Search Results]
* Ottawa Public Library *
Contact Us > Main Branch
Located at: &nbsp;150 Elgin Street, Ottawa
Status: OPEN NOW! 📚
Customer Service: library@ottawa.ca
Visit us online: https://biblioottawalibrary.ca
Postal: K1P 1L7
---------------------
<div class="business-listing">
Sarah's Bakery & Café [⭐️4.8]
Address line 1: 240 
Address line 2: Laurier Avenue
City: Ottawa
Postal Code: K1P 5J7
Contact: sarah@sarahsbakery.ca
https://www.sarahsbakery.ca
</div>
...Read More...
---------------------
TECH CORP GLOBAL
www.techcorp.com/contact
📍 1385 Terry Fox Drive
Kanata, Ontario
[Click to view map]
Email: info@techcorp.com
Postal: K2K 3K2
"""

response = chat(
  model='llama3.1:8b',
  messages=[{'role': 'user', 'content': system_prompt},
           {'role': 'user', 'content': data}],
  format=BusinessLocations.model_json_schema(),  # Use Pydantic to generate the schema or format=schema
  options={'temperature': 0},  # Make responses more deterministic
)


# Use Pydantic to validate the response
data = BusinessLocations.model_validate_json(response.message.content)
pp(data)

BusinessLocations(addresses=[BusinessLocation(name='Ottawa Public Library', street_number='150', street_name='Elgin Street', city='Ottawa', postal_code='K1P 1L7', email='library@ottawa.ca', website='https://biblioottawalibrary.ca'), BusinessLocation(name="Sarah's Bakery & Café", street_number='240', street_name='Laurier Avenue', city='Ottawa', postal_code='K1P 5J7', email='sarah@sarahsbakery.ca', website='https://www.sarahsbakery.ca'), BusinessLocation(name='TECH CORP GLOBAL', street_number='1385', street_name='Terry Fox Drive', city='Kanata, Ontario', postal_code='K2K 3K2', email='info@techcorp.com', website='www.techcorp.com/contact')])


In [27]:
import pandas as pd

def locations_to_df(business_locations: BusinessLocations) -> pd.DataFrame:
    """
    Convert BusinessLocations Pydantic model to pandas DataFrame.
    Each row represents one business location with all its fields.
    """
    # Convert each BusinessLocation to a dict and create DataFrame
    return pd.DataFrame([loc.model_dump() for loc in business_locations.addresses])
# Assuming we have our parsed response in parsed_locations
df = locations_to_df(data)
df

,name,street_number,street_name,city,postal_code,email,website
0,Ottawa Public Library,150,Elgin Street,Ottawa,K1P 1L7,library@ottawa.ca,https://biblioottawalibrary.ca
1,Sarah's Bakery & Café,240,Laurier Avenue,Ottawa,K1P 5J7,sarah@sarahsbakery.ca,https://www.sarahsbakery.ca
2,TECH CORP GLOBAL,1385,Terry Fox Drive,"Kanata, Ontario",K2K 3K2,info@techcorp.com,www.techcorp.com/contact


In [25]:
# Create a more detailed prompt
system_prompt = """
Extract business location information from the following text. For each business, extract:
- name: Business name
- street_number: Building number
- street_name: Street name
- city: City name
- postal_code: Full postal code if present (format: A1A 1A1)
- email: Complete email address
- website: Full website URL (including https://)

Be sure to:
1. Extract ALL available information, particularly postal codes and websites
2. Include the full postal code when it's present (e.g., K1P 1L7)
3. Include complete website URLs when available
4. Extract email addresses exactly as shown

Text to process:
"""


# Make the API call with separate messages
response = chat(
    model='llama3.1:8b',
    messages=[
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': data}
    ],
    format=BusinessLocations.model_json_schema(),
    options={'temperature': 0}
)

ValidationError: 1 validation error for Message
content
  Input should be a valid string [type=string_type, input_value=BusinessLocations(address...rp.com', website=None)]), input_type=BusinessLocations]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type

In [34]:
from enum import Enum
from typing import Optional
from pydantic import BaseModel

text = """
The Ottawa Public Library is located at 120 Metcalfe Street, Ottawa K1P 5M2. This is a large public building.

Sarah Wilson and her family live in unit 4B at 250 Laurier Avenue, Ottawa K1P 6M7.

TechCorp's new headquarters are at 1385 Bank Street in Ottawa K1H 7M5, located in the South Keys Business Park building.

The Rideau Centre mall can be found at 50 Rideau Street, Ottawa K1N 9J7.

Dr. Jones runs his family practice from Suite 302 at 1919 Riverside Drive, Ottawa K1H 1A2. The medical building is called River Medical Centre.

The Blue Condo tower at 100 Bronson Ave has several units for rent, including apartment 2505. The postal code is K1R 7G6.
"""



prompt = """
You are address parsing assistant. 
User provides text containing addresses.
Extract addresses from the text.
Use the following JSON schema to generate your response:


Format as JSON matching this example:
{
    "addresses": [
        {
            "name": "string (business or person name)",
            "street_number": "string (numbers only)",
            "street_name": "string",
            "city": "string",
            "postal_code": "string (Canadian format)",
            "address_type": "residential or commercial",
            "unit_number": "string or null",
            "building_name": "string or null"
        }
    ]
}
Rules:
- Classify buildings like libraries, malls, and offices as commercial
- Houses and apartments as residential
- Include unit numbers if mentioned
- Include building names if mentioned

"""

class AddressType(str, Enum):
    RESIDENTIAL = "residential"
    COMMERCIAL = "commercial"
    
    def requires_business_license(self) -> bool:
        return self == AddressType.COMMERCIAL

class Address(BaseModel):
    name: str
    street_number: str
    street_name: str
    city: str
    postal_code: str
    address_type: AddressType
    unit_number: Optional[str] = None
    building_name: Optional[str] = None


class AddressResponse(BaseModel):
    addresses: list[Address]

response = chat(
  model='llama3.1:8b',
  messages=[{'role': 'user', 'content': prompt},
           {'role': 'user', 'content': text}],
  format=AddressResponse.model_json_schema(),  # Use Pydantic to generate the schema or format=schema
  options={'temperature': 0},  # Make responses more deterministic
)


# Use Pydantic to validate the response
data = AddressResponse.model_validate_json(response.message.content)
pp(data)

AddressResponse(addresses=[Address(name='Ottawa Public Library', street_number='120', street_name='Metcalfe Street', city='Ottawa', postal_code='K1P 5M2', address_type=<AddressType.COMMERCIAL: 'commercial'>, unit_number=None, building_name=None), Address(name='Sarah Wilson', street_number='250', street_name='Laurier Avenue', city='Ottawa', postal_code='K1P 6M7', address_type=<AddressType.RESIDENTIAL: 'residential'>, unit_number='4B', building_name=None), Address(name='TechCorp', street_number='1385', street_name='Bank Street', city='Ottawa', postal_code='K1H 7M5', address_type=<AddressType.COMMERCIAL: 'commercial'>, unit_number=None, building_name='South Keys Business Park'), Address(name='The Rideau Centre mall', street_number='50', street_name='Rideau Street', city='Ottawa', postal_code='K1N 9J7', address_type=<AddressType.COMMERCIAL: 'commercial'>, unit_number=None, building_name=None), Address(name='Dr. Jones', street_number='1919', street_name='Riverside Drive', city='Ottawa', pos

In [35]:
import pandas as pd

def locations_to_df(business_locations: BusinessLocations) -> pd.DataFrame:
    """
    Convert BusinessLocations Pydantic model to pandas DataFrame.
    Each row represents one business location with all its fields.
    """
    # Convert each BusinessLocation to a dict and create DataFrame
    return pd.DataFrame([loc.model_dump() for loc in business_locations.addresses])
# Assuming we have our parsed response in parsed_locations
df = locations_to_df(data)
df

,name,street_number,street_name,city,postal_code,address_type,unit_number,building_name
0,Ottawa Public Library,120,Metcalfe Street,Ottawa,K1P 5M2,AddressType.COMMERCIAL,None,None
1,Sarah Wilson,250,Laurier Avenue,Ottawa,K1P 6M7,AddressType.RESIDENTIAL,4B,None
2,TechCorp,1385,Bank Street,Ottawa,K1H 7M5,AddressType.COMMERCIAL,None,South Keys Business Park
3,The Rideau Centre mall,50,Rideau Street,Ottawa,K1N 9J7,AddressType.COMMERCIAL,None,None
4,Dr. Jones,1919,Riverside Drive,Ottawa,K1H 1A2,AddressType.RESIDENTIAL,None,River Medical Centre
5,The Blue Condo tower,100,Bronson Ave,Ottawa,K1R 7G6,AddressType.RESIDENTIAL,2505,None
